# Dependences

In [1]:
using CUDA
using DelimitedFiles
using Test
using BenchmarkTools

# initial data
R_Agg = 13
r_max = 3.0
n_knn = 50
idx_red_size =  r_max ≤ 2.80 ? 13 :
            2.80 < r_max ≤ 3.45 ? 21 :
            3.45 < r_max ≤ 3.80 ? 39 :
            3.80 < r_max ≤ 4.00 ? 55 :
            70
X = Float64.(readdlm("../../../data/init/Sphere/$(R_Agg).xyz")[3:end,2:end]) |> cu

# kNN
idx      = Int32.(zeros(size(X, 1), size(X, 1))) |> cu;
idx_sum  = Int32.(zeros(1, size(idx, 1))) |> cu
idx_red  = Int32.(zeros(idx_red_size, size(idx, 1))) |> cu
idx_contractile =Int32.(zeros(n_knn,size(X,1))) |> cu

# forces
force = CUDA.zeros(size(X));

# Generic Functions

In [2]:
# force_func
cubic(A,r_max,r_min,dist) = - A * (dist-r_max)^2* (dist-r_min)

# distances
euclidean(points,i,j) = sqrt((points[i,1]-points[j,1])^2+(points[i,2]-points[j,2])^2+(points[i,3]-points[j,3])^2)

euclidean (generic function with 1 method)

# idx

## functions

In [3]:
function dist_kernel!(idx, points ,r_max)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    
    # Limiting data inside matrix
    if i <= size(points, 1) && j <= size(points, 1)
        if euclidean(points,i,j) < r_max
            idx[i, j] = i
        else
            idx[i, j] = 0
        end 
    end
    return nothing
end

function reduce_kernel(idx,idx_red,idx_sum)
    # Defining Index for kernel
    i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

    # Limiting data inside matrix
    if i <= size(idx,1)
        # Cleaning idx_sum
        idx_sum[i] = 0
        
        # looping on each row for searching non-zero values
        for j = 1:size(idx,1)
            if idx[j,i] != 0
                idx_sum[i] += 1
                idx_red[idx_sum[i],i] = j
            end
        end
    end
    
    return nothing
end

function index_contractile!(idx_contractile,idx_sum,idx_red)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(idx_contractile, 1) && j <= size(idx_contractile,2)
        idx_contractile[i,j] = idx_red[rand(1:idx_sum[j]),j]
    end
    return nothing
end

function nearest_neighbors(idx, idx_red, idx_sum, idx_contractile, points ,r_max)
    # Calculating Distance Matrix
    threads =(32,32)
    blocks  =cld.(size(points,1),threads)
    @cuda threads=threads blocks=blocks dist_kernel!(idx, points ,r_max)

    # Reducing Distance Matrix to Nearest Neighbors
    threads=1024
    blocks=cld.(size(idx,1),threads)
    @cuda threads=threads blocks=blocks reduce_kernel(idx,idx_red,idx_sum)

    # Finding index contractile
    threads =(32,32)
    blocks  =cld.(size(X,1),threads)
    @cuda threads=threads blocks=blocks index_contractile!(idx_contractile,idx_sum,idx_red)
end

nearest_neighbors (generic function with 1 method)

## running

In [15]:
println("---------------------------------- VARIABLES ------------------------------------")
println("R_Agg = $(R_Agg) | R_Max = $(r_max) | col_size_idx = $(idx_red_size)")
display(X)
println("------------------------------------ SIZES --------------------------------------")
println("Size → idx     = $(size(idx))")
println("     → idx_sum = $(size(idx_sum))")
println("     → idx_red = $(size(idx_red))")
println("------------------------------------ RESULTS ------------------------------------")
@btime nearest_neighbors(idx, idx_red, idx_sum,idx_contractile, X ,r_max)
println("----------------------------------- IDX RED -------------------------------------")
display(idx_red)
println("----------------------------------- IDX SUM -------------------------------------")
display(idx_sum)
println("------------------------------- IDX CONTRACTILE ---------------------------------")
display(idx_contractile)
println("---------------------------------- Memory Used ----------------------------------")
CUDA.memory_status()  

---------------------------------- VARIABLES ------------------------------------
R_Agg = 13 | R_Max = 3.0 | col_size_idx = 21


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.5  -4.04  -12.25
  1.5  -4.04  -12.25
 -3.5  -2.31  -12.25
 -1.5  -2.31  -12.25
  0.5  -2.31  -12.25
  2.5  -2.31  -12.25
 -2.5  -0.58  -12.25
 -0.5  -0.58  -12.25
  1.5  -0.58  -12.25
  3.5  -0.58  -12.25
 -3.5   1.15  -12.25
 -1.5   1.15  -12.25
  0.5   1.15  -12.25
  ⋮           
  0.5  -1.15   12.25
  2.5  -1.15   12.25
 -2.5   0.58   12.25
 -0.5   0.58   12.25
  1.5   0.58   12.25
  3.5   0.58   12.25
 -3.5   2.31   12.25
 -1.5   2.31   12.25
  0.5   2.31   12.25
  2.5   2.31   12.25
 -0.5   4.04   12.25
  1.5   4.04   12.25

------------------------------------ SIZES --------------------------------------
Size → idx     = (1620, 1620)
     → idx_sum = (1, 1620)
     → idx_red = (21, 1620)
------------------------------------ RESULTS ------------------------------------
  13.420 μs (77 allocations: 4.03 KiB)
----------------------------------- IDX RED -------------------------------------


21×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  1   1   3   1   1   2   3   4   5  …  1580  1581  1582  1583  1589  1590
  2   2   4   3   2   5   4   5   6     1581  1582  1583  1584  1590  1591
  4   5   7   4   4   6   7   7   8     1582  1583  1584  1585  1591  1592
  5   6  23   5   5   9   8   8   9     1588  1589  1590  1591  1596  1597
 19  20  29   7   6  10  11   9  10     1589  1590  1591  1592  1597  1598
 24  25  30   8   8  26  12  12  13  …  1595  1596  1597  1598  1601  1602
 25  26  35  24   9  32  30  13  14     1611  1611  1612  1613  1616  1617
 30  31  36  30  25  33  36  31  32     1615  1612  1613  1614  1617  1618
 31  32  37  31  31  38  37  37  38     1616  1615  1616  1617  1619  1619
 32  33   0  36  32  39  43  38  39        0  1616  1617  1618  1620  1620
  0   0   0  37  37  40  44  44  45  …     0  1617  1618  1620     0     0
  0   0   0  38  38   0  45  45  46        0  1619  1619     0     0     0
  0   0   0   0  39   0   0  46  47        0     0

----------------------------------- IDX SUM -------------------------------------


1×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 10  10  9  12  13  11  12  13  13  10  …  13  13  10  9  12  13  11  10  10

------------------------------- IDX CONTRACTILE ---------------------------------


50×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 24   1  37  37   1  38  11  37   8  …  1580  1619  1620  1614  1601  1591
 19  33  30  36   9   2   8   9  47     1581  1619  1617  1617  1619  1602
  1  32   4  37  39  33  30  38   6     1580  1619  1582  1585  1590  1618
  1  32  35  30   1  38  30  38  47     1588  1590  1612  1617  1591  1597
 31  26  23  30  37   2  37  13   6     1581  1615  1620  1613  1597  1598
 19  26   4   8  31  26  45  46  47  …  1588  1617  1619  1620  1617  1591
  2   5  29  30  39  32  30   7  32     1588  1581  1597  1592  1589  1619
 25  33  30  37   9   2  36  45  47     1615  1583  1620  1584  1617  1590
 19  31  29   4  25  39  30  44   8     1589  1612  1619  1592  1590  1620
  5   2  30   1   2   6   8  38  10     1581  1615  1612  1614  1591  1592
 24   1  36  38   6  40  44  45  45  …  1595  1583  1597  1591  1601  1617
 32  31  35   4  38   5   8  38  10     1615  1583  1612  1591  1589  1590
 19  32  29  38   5   6  30   9  38     1580  1589

---------------------------------- Memory Used ----------------------------------
Effective GPU memory usage: 27.22% (544.938 MiB/1.955 GiB)
No memory pool is in use.

# forces

## functions

In [5]:
function sum_force!(idx,idx_contractile,points,force, force_func,A,r_max,r_min,A_con,t_knn)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(points, 1) && k <= size(points, 2)

        # Cleaning idx_sum
        force[i,k] = 0
        dist = 0

        # Iterate on each row
        for j=1:size(idx,1)

            # Finding forces
            if idx[j,i] != i && idx[j,i] != 0
                dist = euclidean(points,i,idx[j,i])
                force[i,k] +=force_func(A,r_max,r_min,dist) * points[idx[j,i],k] / dist
            end
            
        end

        # Adding Contrractile Force
        if idx_contractile[t_knn,i] != i && idx_contractile[t_knn,i] != 0
            dist = euclidean(points,i,idx_contractile[t_knn,i])
            force[i,k] += A_con*points[idx_contractile[t_knn,i],k]/dist
        end
        
    end
    return nothing
end

sum_force! (generic function with 1 method)

## running

In [6]:
dt = 0.5
A     = 0.1
A_con = 0.001
r_min = 2.0
r_max = r_max

t_knn = 1

1

In [16]:
# GPU
threads =(100,3)
blocks  =cld.(size(X,1),threads)
@cuda threads=threads blocks=blocks sum_force!(idx_red,idx_contractile, X ,force, cubic,A,r_max,r_min,A_con,Int(t_knn))

CUDA.HostKernel{typeof(sum_force!), Tuple{CuDeviceMatrix{Int32, 1}, CuDeviceMatrix{Int32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, typeof(cubic), Float64, Float64, Float64, Float64, Int64}}(sum_force!, CuFunction(Ptr{CUDA.CUfunc_st} @0x0000000008f38870, CuModule(Ptr{CUDA.CUmod_st} @0x00000000058b5e20, CuContext(0x0000000003f6d960, instance dd3a13daea393c81))), CUDA.KernelState(Ptr{Nothing} @0x0000000203400000))

## Pending (Test with CPU)

In [17]:
# CPU
X_CPU = Matrix(X)
idx_red_CPU = Matrix(idx_red)
idx_contractile_CPU = Matrix(idx_contractile)
force_CPU = zeros(size((X)))


for j = 1:size(idx_red_CPU,1)
    for i =1:size(X,1)
        if idx_red_CPU[j,i] != 0 && idx_red_CPU[j,i] != i
            dist = euclidean(X_CPU,i,idx_red_CPU[j,i])
            force_CPU[i,:] += cubic.(A,r_max,r_min,dist) .* X_CPU[idx_red_CPU[j,i],:] ./ dist
        end
    end
end

for i =1:size(X,1)
    if idx_contractile_CPU[t_knn,i] != i
        dist = euclidean(X_CPU,i,idx_contractile_CPU[t_knn,i])
        force_CPU[i,:] += A_con .* X_CPU[idx_contractile_CPU[t_knn,i],:] ./ dist
    end
end

In [18]:
# testing
@test Matrix(force) ≈ force_CPU atol=0.01

Test Passed

In [19]:
display(force)
display(force_CPU)

1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.000810042   0.0107469     0.0274723
 -0.00492285    0.0110295     0.0266351
  0.0114927     0.00752344    0.0322838
  0.00418892    0.00694463    0.02856
 -0.00182112    0.00513503    0.0266189
 -0.00799023    0.00711142    0.0300989
  0.00540399    0.00196914    0.0236526
  0.000682465   0.000819668   0.0244877
 -0.00454333    0.00110363    0.0236581
 -0.00957266    0.000923119   0.0266726
  0.0114047    -0.00434438    0.0318859
  0.00544088   -0.00450937    0.0349612
 -0.00114925   -0.00450937    0.0349612
  ⋮                          
 -0.00114925    0.00450937   -0.0349612
 -0.00741355    0.00537821   -0.0331624
  0.00690554   -0.00110363   -0.0236581
  0.000680442  -0.000237602  -0.0236526
 -0.00304559   -0.000819668  -0.0244877
 -0.00956929   -0.00207259   -0.0258376
  0.0100816    -0.00752344   -0.0322838
  0.00468542   -0.00549532   -0.0277114
 -0.000820229  -0.00513503   -0.0266189
 -0.00641506   -0.00722686   -0.0277114
 

1620×3 Matrix{Float64}:
  0.000810042   0.0107469     0.0274723
 -0.00492285    0.0110295     0.0266351
  0.0114927     0.00752344    0.0322838
  0.00418892    0.00694463    0.02856
 -0.00182112    0.00513503    0.0266189
 -0.00799022    0.00711142    0.0300989
  0.00540399    0.00196914    0.0236527
  0.000682465   0.000819668   0.0244877
 -0.00454333    0.00110363    0.0236581
 -0.00957266    0.000923119   0.0266726
  0.0114047    -0.00434438    0.0318859
  0.00544088   -0.00450937    0.0349612
 -0.00114925   -0.00450937    0.0349612
  ⋮                          
 -0.00114925    0.00450937   -0.0349612
 -0.00741356    0.00537821   -0.0331624
  0.00690554   -0.00110363   -0.0236581
  0.000680442  -0.000237602  -0.0236527
 -0.00304559   -0.000819668  -0.0244877
 -0.00956929   -0.00207259   -0.0258376
  0.0100816    -0.00752344   -0.0322838
  0.00468542   -0.00549532   -0.0277114
 -0.000820229  -0.00513503   -0.0266189
 -0.00641506   -0.00722686   -0.0277114
  0.00138112   -0.010808    